In [13]:
#models/DiffSynth-Studio/Qwen-Image-Blockwise-ControlNet-Inpaint/model.safetensors

import torch
from safetensors import safe_open

# 使用 safetensors 库来加载 .safetensors 文件
with safe_open("train/Qwen-Image-Edit-2509_inpaint_controlnet_and_lora/step-1.safetensors", framework="pt", device="cpu") as f:
    model = {}
    for k in f.keys():
        model[k] = f.get_tensor(k)

print("模型中的键:")
for key in model.keys():
    print(f"- {key}: {model[key].shape}")

/data/student/s2430069/.local/share/mamba/envs/diffy/lib/python3.13/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


模型中的键:
- pipe.blockwise_controlnet.models.0.controlnet_blocks.0.input_proj.bias: torch.Size([3072])
- pipe.blockwise_controlnet.models.0.controlnet_blocks.0.input_proj.weight: torch.Size([3072, 3072])
- pipe.blockwise_controlnet.models.0.controlnet_blocks.0.output_proj.bias: torch.Size([3072])
- pipe.blockwise_controlnet.models.0.controlnet_blocks.0.output_proj.weight: torch.Size([3072, 3072])
- pipe.blockwise_controlnet.models.0.controlnet_blocks.0.x_rms.weight: torch.Size([3072])
- pipe.blockwise_controlnet.models.0.controlnet_blocks.0.y_rms.weight: torch.Size([3072])
- pipe.blockwise_controlnet.models.0.controlnet_blocks.1.input_proj.bias: torch.Size([3072])
- pipe.blockwise_controlnet.models.0.controlnet_blocks.1.input_proj.weight: torch.Size([3072, 3072])
- pipe.blockwise_controlnet.models.0.controlnet_blocks.1.output_proj.bias: torch.Size([3072])
- pipe.blockwise_controlnet.models.0.controlnet_blocks.1.output_proj.weight: torch.Size([3072, 3072])
- pipe.blockwise_controlnet.model

In [8]:
# 修改 img_in.weight 的形状
if "img_in.weight" in model:
    print(f"原始 img_in.weight 形状: {model['img_in.weight'].shape}")
    
    # 删除原来的权重
    del model["img_in.weight"]
    
    # 创建新的权重张量，形状为 [3072, 324]
    new_weight = torch.randn(3072, 320, dtype=torch.float32)
    model["img_in.weight"] = new_weight
    
    print(f"新的 img_in.weight 形状: {model['img_in.weight'].shape}")
else:
    print("未找到 img_in.weight")

# 显示修改后的模型键信息
print("\n修改后的模型中的键:")
for key in model.keys():
    print(f"- {key}: {model[key].shape}")
#保存为models/DiffSynth-Studio/Qwen-Image-Blockwise-ControlNet-Inpaint/model.safetensors
from safetensors.torch import save_file
save_file(model, "models/DiffSynth-Studio/Qwen-Image-Blockwise-ControlNet-Inpaint/model.safetensors")


原始 img_in.weight 形状: torch.Size([3072, 324])
新的 img_in.weight 形状: torch.Size([3072, 320])

修改后的模型中的键:
- controlnet_blocks.0.input_proj.bias: torch.Size([3072])
- controlnet_blocks.0.input_proj.weight: torch.Size([3072, 3072])
- controlnet_blocks.0.output_proj.bias: torch.Size([3072])
- controlnet_blocks.0.output_proj.weight: torch.Size([3072, 3072])
- controlnet_blocks.0.x_rms.weight: torch.Size([3072])
- controlnet_blocks.0.y_rms.weight: torch.Size([3072])
- controlnet_blocks.1.input_proj.bias: torch.Size([3072])
- controlnet_blocks.1.input_proj.weight: torch.Size([3072, 3072])
- controlnet_blocks.1.output_proj.bias: torch.Size([3072])
- controlnet_blocks.1.output_proj.weight: torch.Size([3072, 3072])
- controlnet_blocks.1.x_rms.weight: torch.Size([3072])
- controlnet_blocks.1.y_rms.weight: torch.Size([3072])
- controlnet_blocks.10.input_proj.bias: torch.Size([3072])
- controlnet_blocks.10.input_proj.weight: torch.Size([3072, 3072])
- controlnet_blocks.10.output_proj.bias: torch.Size(

In [10]:
import json

json_path = "prepared_data_original/metadata.json"

with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

for item in data:
    if item.get("ref_file"):
        item["ref_file"] = f"ref/{item['ref_file']}"
    if item.get("tgt_original_file"):
        item["tgt_original_file"] = f"tgt_original/{item['tgt_original_file']}"
    if item.get("tgt_original_mask_file"):
        item["tgt_original_mask_file"] = f"tgt_original/{item['tgt_original_mask_file']}"

with open(json_path, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print("图片路径已批量修正为 ref/ 和 tgt_original/ 子目录。")

图片路径已批量修正为 ref/ 和 tgt_original/ 子目录。


In [11]:
import os
import json

json_path = "prepared_data_original/metadata.json"
base_ref = "prepared_data_original/ref"
base_tgt = "prepared_data_original/tgt_original"

with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 统计有效图片数量
valid_ref = 0
valid_tgt = 0
for item in data:
    if item.get("ref_file"):
        ref_path = os.path.join(base_ref, item["ref_file"].split("/")[-1])
        if os.path.isfile(ref_path):
            valid_ref += 1
            item["ref_file"] = f"ref/{item['ref_file'].split('/')[-1]}"
        else:
            item["ref_file"] = None
    if item.get("tgt_original_file"):
        tgt_path = os.path.join(base_tgt, item["tgt_original_file"].split("/")[-1])
        if os.path.isfile(tgt_path):
            valid_tgt += 1
            item["tgt_original_file"] = f"tgt_original/{item['tgt_original_file'].split('/')[-1]}"
        else:
            item["tgt_original_file"] = None
    if item.get("tgt_original_mask_file"):
        mask_path = os.path.join(base_tgt, item["tgt_original_mask_file"].split("/")[-1])
        if os.path.isfile(mask_path):
            item["tgt_original_mask_file"] = f"tgt_original/{item['tgt_original_mask_file'].split('/')[-1]}"
        else:
            item["tgt_original_mask_file"] = None

# 删除无效图片的条目
data = [item for item in data if item.get("ref_file") and item.get("tgt_original_file")]

with open(json_path, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print(f"有效ref图片数量: {valid_ref}")
print(f"有效tgt_original图片数量: {valid_tgt}")
print("无效图片条目已删除，路径已修正。")

KeyboardInterrupt: 

In [8]:
import os
import json
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

json_path = "prepared_data_original/metadata.json"
base_ref = "prepared_data_original/ref"
base_tgt = "prepared_data_original/tgt_original"

with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

def check_and_fix(item):
    result = item.copy()
    valid_ref = False
    valid_tgt = False
    # 检查ref图片
    if item.get("ref_file"):
        ref_name = item["ref_file"].split("/")[-1]
        ref_path = os.path.join(base_ref, ref_name)
        if os.path.isfile(ref_path):
            result["ref_file"] = f"ref/{ref_name}"
            valid_ref = True
        else:
            result["ref_file"] = None
    # 检查tgt图片
    if item.get("tgt_original_file"):
        tgt_name = item["tgt_original_file"].split("/")[-1]
        tgt_path = os.path.join(base_tgt, tgt_name)
        if os.path.isfile(tgt_path):
            result["tgt_original_file"] = f"tgt_original/{tgt_name}"
            valid_tgt = True
        else:
            result["tgt_original_file"] = None
    # 检查mask图片
    if item.get("tgt_original_mask_file"):
        mask_name = item["tgt_original_mask_file"].split("/")[-1]
        mask_path = os.path.join(base_tgt, mask_name)
        if os.path.isfile(mask_path):
            result["tgt_original_mask_file"] = f"tgt_original/{mask_name}"
        else:
            result["tgt_original_mask_file"] = None
    return result, valid_ref, valid_tgt

new_data = []
valid_ref = 0
valid_tgt = 0
with ThreadPoolExecutor(max_workers=64) as executor:
    futures = [executor.submit(check_and_fix, item) for item in data]
    for fut in tqdm(as_completed(futures), total=len(futures), desc="检查图片"):
        item, ref_ok, tgt_ok = fut.result()
        if item.get("ref_file") and item.get("tgt_original_file"):
            new_data.append(item)
        if ref_ok: valid_ref += 1
        if tgt_ok: valid_tgt += 1

with open(json_path, "w", encoding="utf-8") as f:
    json.dump(new_data, f, ensure_ascii=False, indent=2)

print(f"有效ref图片数量: {valid_ref}")
print(f"有效tgt_original图片数量: {valid_tgt}")
print("无效图片条目已删除，路径已修正。")

检查图片: 100%|██████████| 111767/111767 [01:11<00:00, 1570.22it/s]



有效ref图片数量: 111767
有效tgt_original图片数量: 111767
无效图片条目已删除，路径已修正。


In [12]:
len(new_data)

111767